In [ ]:
import cv2
import numpy as np

img = cv2.imread('238_original.jpg', cv2.IMREAD_COLOR)

dst_img = np.zeros_like(img)

camera_matrix = np.array([
    [4208, 0, 2104],
    [0, 4208, 1560],
    [0, 0, 1]
])
dist_coeffs = np.array([
    -0.667539, #k1
    -0.608235, #k2
    0.015444, #p1
    0.003268, #p2
    1.8
])
# dist_coeffs = np.array([0,0,0,0])

corrected_img = cv2.undistort(img, camera_matrix, dist_coeffs)

cv2.imwrite('out.jpg', corrected_img)


In [ ]:
import cameratransform as ct
import cv2
import numpy as np
from typing import Dict

geo_ref_points = [
    (52.42413653059658, 10.78672906663592, 0),
    (52.42458455040172, 10.786824824162142, 0),
    (52.42451227298838, 10.78703369789507, 0),
    (52.4245142736079, 10.787052467703178, 0),
    (52.42455500631099, 10.78721166168216, 0),
    (52.424360588305404, 10.787260492574516, 0),
    (52.424285428844534, 10.78705339553127, 0),
]

camera_params = {
    "244": {
        "elevation_m": 10,
        "heading_deg": 39.5,
        "image_height_px": 960,
        "image_width_px": 1280,
        "pos_lat": 52.42415872,
        "pos_lon": 10.78691456,
        "tilt_deg": 43.5,
        "view_x_deg": 92,
        "view_y_deg": 71,
        "brown_distortion_k1": -0.19,
        "brown_distortion_k2": 0.035,
    },
    "238": {
        "elevation_m": 10,
        "heading_deg": 317,
        "image_height_px": 960,
        "image_width_px": 1280,
        "pos_lat": 52.42415872,
        "pos_lon": 10.78691456,
        "tilt_deg": 49,
        "view_x_deg": 88,
        "view_y_deg": 65,
        "brown_distortion_k1": -0.24,
        "brown_distortion_k2": 0.045,
    }
}

cameras: Dict[str, ct.Camera] = {}

top_views = {}

for name, params in camera_params.items():
    cam = ct.Camera(
        projection=ct.RectilinearProjection(
            image_height_px=params['image_height_px'],
            image_width_px=params['image_width_px'],
            view_x_deg=params['view_x_deg'],
            view_y_deg=params['view_y_deg']
        ),
        orientation=ct.SpatialOrientation(
            elevation_m=params['elevation_m'],
            heading_deg=params['heading_deg'],
            tilt_deg=params['tilt_deg']
        ),
        lens=ct.BrownLensDistortion(
            k1=params['brown_distortion_k1'],
            k2=params['brown_distortion_k2'],
        )
    )
    cam.setGPSpos(params['pos_lat'], params['pos_lon'])

    cameras[name] = cam

    img = cv2.imread(f'{name}_original.jpg', cv2.IMREAD_COLOR)
    
    top_view = cam.getTopViewOfImage(
        img,
        extent=(-100,100,-100,100),
        scaling=0.05
    )
    top_views[name] = top_view

    # cv2.imwrite(f'{cam_name}_ct_undistort.jpg', cam.undistortImage(img))
    cv2.imwrite(f'{name}_ct_mapped.jpg', top_view)

ref_cam = next(iter(cameras.values()))
space_ref_points = ref_cam.spaceFromGPS(np.array(geo_ref_points))
print(space_ref_points)
space_ref_points[:,1] *= -1
space_ref_points[:,:2] += 100
space_ref_points[:,:2] *= 20

merged = next(iter(top_views.values()))
merged = cv2.addWeighted(top_views['238'], 0.5, top_views['244'], 0.5, 0.0)
diff = cv2.absdiff(top_views['238'], top_views['244'])
for point in space_ref_points:
    merged = cv2.circle(merged, (int(point[0]), int(point[1])), 5, color=(0, 0, 255), thickness=-1)
cv2.imwrite(f'merged.jpg', merged)
cv2.imwrite(f'diff.jpg', diff)


In [ ]:
space_ref_points